# Strategy

Input: Location in lat and long

Process: Find the 10 nearest neighbors from the lat and long and take the average of that location. 
That is the feature 

Output: that particular feature. 

# Preparing the Data

In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
combined_data = pd.read_hdf('../data/d_success',)

In [4]:
businesses = pd.read_hdf('../data/bus_with_sucess.hdf')

In [ ]:
train_bus, test_bus = 

- Calculate the geographic distance away from everything with shapely

# Distance testing

In [6]:
from geopy.distance import vincenty

In [18]:
businesses = businesses.reset_index()

In [19]:
businesses.ix[1]

index                                                             3
business_id                                  mVHrayjG3uZ_RLHkLj-AMg
categories        [Bars, American (New), Nightlife, Lounges, Res...
city                                                       Braddock
full_address                    414 Hawkins Ave\nBraddock, PA 15104
hours             {'Saturday': {'open': '10:00', 'close': '16:00...
latitude                                                    40.4087
longitude                                                  -79.8664
name                                                  Emil's Lounge
neighborhoods                                                    []
open                                                           True
review_count                                                     11
stars                                                           4.5
state                                                            PA
type                                            

In [23]:
loc1 = (businesses.ix[1]['latitude'], businesses.ix[1]['longitude'])
loc2 = (businesses.ix[5]['latitude'], businesses.ix[5]['longitude'])

In [28]:
from scipy.spatial import cKDTree as KDTree

In [31]:
loc_cols = ['latitude', 'longitude']
tree = KDTree(businesses[loc_cols])

In [54]:
distance, indices = tree.query(businesses[loc_cols], 6)

In [44]:
?tree.query

In [36]:
indices

array([    0,     1,     2, ..., 22755, 22756, 22757])

## Given a Point

In [107]:
point = (40.415519, -80.067534)

In [108]:
distance, indices = tree.query(point, 6)

In [109]:
distance

array([  2.00000000e-06,   5.95393962e-04,   5.63332509e-03,
         6.31602945e-03,   1.53749148e-02,   1.56933918e-02])

In [110]:
businesses.ix[indices].success_metric.mean()

0.6200550324692973

In [82]:
def point_find_nearest_neighbors(df, point, k=5, loc_cols=['latitude', 'longitude']):
    """
    Given a point (lat, long) 
    :param df: 
    :param point: 
    :param k: 
    :param loc_cols: 
    :return: 
    """
    tree = KDTree(df[loc_cols])
    distance, indices = tree.query(point, k)
    return df.ix[indices]

In [118]:
def row_find_nearest_businesses(df, row, k=5, loc_cols=['latitude', 'longitude']):
    """
    Finds the nearest neighbor of a

    :param row: Row that we are interested in finding the nearest neighbors for
    :param df: pd.DataFrame with loc_cols
    :param k:
    :param loc_cols: List of columns that we are comparing to for nearest neighbors
    :return:

    Example
    ---
    >>> find_nearest_neighbors(businesses, 1)
    """
    tree = KDTree(df[loc_cols])
    distance, indices = tree.query(df[loc_cols], k + 1)
    neighbors = df.ix[indices[row][1:]] # Start at 1 to ignore the current index

    return neighbors.success_metric.mean()

In [129]:
businesses['neighbor_success'] = -1

In [137]:
for i, row in businesses.iterrows():
    businesses.ix[i, 'neighbor_success'] = row_find_nearest_businesses(businesses, i)
    if i % 1000 == 0:
        print('Done with {} entries'.format(i))

Done with 0 entries
Done with 1000 entries
Done with 2000 entries
Done with 3000 entries
Done with 4000 entries
Done with 5000 entries
Done with 6000 entries
Done with 7000 entries
Done with 8000 entries
Done with 9000 entries
Done with 10000 entries
Done with 11000 entries
Done with 12000 entries
Done with 13000 entries
Done with 14000 entries
Done with 15000 entries
Done with 16000 entries
Done with 17000 entries
Done with 18000 entries
Done with 19000 entries
Done with 20000 entries
Done with 21000 entries
Done with 22000 entries


In [139]:
businesses.to_hdf('../data/neighbors_bus.hdf', 'df')

/Users/Will/anaconda/envs/py34/lib/python3.4/site-packages/pandas/core/generic.py:938: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->['business_id', 'categories', 'city', 'full_address', 'hours', 'name', 'neighborhoods', 'state', 'type']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


# References

- http://geoffboeing.com/2014/08/clustering-to-reduce-spatial-data-set-size/